# HaMLeT

## Session 6: Backpropagation
by Leon Weninger and Raphael Kolk

### Goal of this Session

In this session you will, step by step, implement a the backpropagation algorithm yourself without using any deep learning libraries. You should already be familiar with Python as well as NumPy (a package for scientific computing with Python).

### Given code

**Task 0:** Familiarize yourself briefly with the given code. Pay particural attention to the `Layer` and `Cost` classes, from which you will derive the classes you implement, and the `Sigmoid` layer which is predefined as an example. You'll also need to execute the cells in this section once.

The following code loads the data and trains the network in a similar fashion as in the last session.

In [94]:
import numpy as np
from tqdm import tqdm
from load_mnist import MNIST


def vectorize(j):
    label_vector = np.zeros((1, 10))
    label_vector[0, int(j)] = 1.0
    return label_vector


def load_data():
    mnist = MNIST()
    images, labels = mnist.data, mnist.target

    image_size = images.shape[1]
    label_size = labels.shape[1]

    random_permutation = np.random.permutation(images.shape[0])
    images = images[random_permutation, :]
    labels = labels[random_permutation, :]
    
    images = (images - np.mean(images))/np.std(images)

    return images, labels, image_size, label_size


def train(net, cost_function, number_epochs, batch_size, learning_rate):
    images, labels, image_size, label_size = load_data()
    training_images, validation_images = images[:50000], images[50000:]
    training_labels, validation_labels = labels[:50000], labels[50000:]

    for e in range(number_epochs):
        cost = train_epoch(e, net, training_images, training_labels, cost_function, batch_size, learning_rate)
        accuracy = validate_epoch(e, net, validation_images, validation_labels, batch_size)
        print('cost=%5.6f, accuracy=%2.6f' % (cost, accuracy), flush=True)


def train_epoch(e, net, images, labels, cost_function, batch_size, learning_rate):
    epoch_cost = 0
    for i in tqdm(range(0, len(images), batch_size), ascii=False, desc='training,   e=%i' % e):
        
        batch_images = images[i:min(i + batch_size, len(images)), :]
        batch_labels = labels[i:min(i + batch_size, len(labels)), :]
        
        if(batch_images.shape[0] != batch_size):
            break

        
        # zero the gradients
        net.zero_gradients()

        # forward pass
        prediction = net.forward(batch_images)
        cost = cost_function.estimate(batch_labels, prediction)

        # backward pass
        dprediction = cost_function.gradient(cost)
        net.backward(dprediction)

        # update the parameters using the computed gradients via stochastic gradient descent.
        net.update_parameters(learning_rate)

        epoch_cost += np.mean(cost)
        
    return epoch_cost


def validate_epoch(e, net, images, labels, batch_size):
    n_correct = 0
    n_total = 0

    for i in tqdm(range(0, len(images), batch_size), ascii=False, desc='validation, e=%i' % e):
        batch_images = images[i:min(i + batch_size, len(images)), :]
        batch_labels = labels[i:min(i + batch_size, len(labels)), :]
        
        if(batch_images.shape[0] != batch_size):
            break
        # compute predicted probabilities.
        predictions = net.forward(batch_images)

        # find the most probable class label.
        n_correct += sum(np.argmax(batch_labels, axis=1) == np.argmax(predictions, axis=1))
        n_total += batch_labels.shape[0]

    return n_correct / n_total


Remember the sigmoid function and its derivative you implemented in the previous session.

In [83]:
def sigmoid_function(var):
    return 1.0 / (1.0 + np.exp(-var))


def sigmoid_derivative(z):
    return sigmoid_function(z) * (1 - sigmoid_function(z))

The following abstract classes should serve as parent classes for all the different layers and cost functions which you will implement. 

In [84]:
class Layer:
    def __init__(self):
        # Initialize all member variables of the layer.
        pass

    def forward(self, x_in):
        # Implemets for forward pass of the layer and returns x_out.
        pass

    def backward(self, d_out):
        # Implements the backward pass of the layer and returns d_in.
        pass

    def zero_gradients(self):
        # Sets all gradients of the layer to zero.
        pass

    def update_parameters(self, learning_rate):
        # Update the parameters of the layer with the help of the gradients stored during the backward pass.
        pass


class Cost:
    def __init__(self):
        # Initialize all member variables of the cost function.
        pass

    def estimate(self, target, prediction):
        # Estimates and return the cost with respect to the predicted label and a target label previously set by set_target().
        pass

    def gradient(self, cost):
        # Calculates and returns the gradient with respect to the cost.
        pass

The following class derived from the Layer class implements the forward and backward pass of the sigmoid activation function alread known from the previous session and serves as an example for you. Since it does not have learnable parameters, no `update_parameters` or `zero_gradients` function needs to be implemented.

In [85]:
class Sigmoid(Layer):
    def __init__(self):
        self.x_in = None
    
    def forward(self, x_in):
        self.x_in = x_in
        x_out = sigmoid_function(x_in)
        return x_out
    
    def backward(self, d_out):
        d_in = d_out * sigmoid_derivative(self.x_in)
        return d_in
    
    def zero_gradients(self):
        pass
    
    def update_parameters(self, learning_rate):
        pass

### Theoretical Foundation 

**Task 1a:** Take a peace of paper and a pencil, use your knowledge from the preparation material and the introduction slides and fill in the gaps in the preparation material. Having written the formulas down, please check with the tutor if they are correct.

Before starting to work on the code, think about batched forward- and backward passing. We keep to the convention of having the batch as the first dimension of all tensors. This is consistent with modern Deep Learning Frameworks, as you will get to know in the next session. However, this convention may change when tensors needs to be transposed when performing multiplications and additions.

In [86]:
class Linear(Layer):
    def __init__(self, n_in, n_out, initial_sigma=0.1):
        self.n_in = n_in
        self.n_out = n_out
        self.w = initial_sigma * np.random.randn(n_out, n_in)
        self.b = np.zeros((1, n_out))
        self.zero_gradients()
        self.x_in = None

    def forward(self, x_in):
        self.x_in = x_in
        x_out = np.matmul(x_in, self.w.T) + self.b #np.matmul(x_in.T, self.w ) + self.b
        return x_out

    def backward(self, d_out):
        self.d_in = np.dot(d_out,self.w )
        self.db = d_out
        self.dw = np.dot(self.x_in.T, d_out).T
        return self.d_in

    def zero_gradients(self):
        self.dw = np.zeros((self.n_out, self.n_in))
        self.db = np.zeros((1, self.n_out))
        self.dx = np.empty((0, self.n_in))

    def update_parameters(self, learning_rate):
        self.w = self.w - learning_rate * self.dw
        self.b = self.b - learning_rate * self.db


class MeanSquareError(Cost):
    def __init__(self):
        self.prediction = None
        self.target = None

    def estimate(self, target, prediction):
        self.prediction = prediction
        self.target = target
        cost = 0.5 * np.sum( (target-prediction) ** 2) 
        return cost

    def gradient(self, cost):
        gradient = self.prediction - self.target
        return gradient



    
class Network(Layer):
    def __init__(self, layers):
        self.layers = layers

    #Implement the Network class which can encapsulate multiple layers. It offers the same interface as a layer and is therefore derived
    #from the Layer parent as well. Make sure to implement all member functions needed. 
    #The forward function propagates a given input through all encapsulated layers and returns the final prediction of the network, 
    #whereas the backward function propagates a given gradient through all layers in reversed order. 
    #zero_gradients and update_parameters invoke the respective functions of the encapsulated layers.
    
    def forward(self,x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, d):
        for layer in reversed(self.layers):
            d = layer.backward(d)
        return d
    
    def zero_gradients(self):
        for layer in self.layers:
            layer.zero_gradients()
    
    def update_parameters(self, learning_rate):
        for layer in self.layers:
            layer.update_parameters(learning_rate)

In [103]:
# define hyperparameters
input_size = 28**2
label_size = 10
batch_size = 600
learning_rate = 0.0001
number_epochs = 100

random_input = np.random.rand(batch_size, input_size)
random_label = np.random.rand(batch_size, label_size)

linear_layer = Linear(input_size, label_size)
sigmoid_layer = Sigmoid()
cost_function = MeanSquareError()

# Test your implementation by propagating random input through a linear layer followed by a sigmoid layer, estimating the mean square error to a random target, 
# calculating the gradient and propagating it back through the sigmoid and linear layer. Afterwards update the parameters of the linear layer using the function you implemented.
z = linear_layer.forward(random_input)
a = sigmoid_layer.forward(z)
cost = cost_function.estimate(random_label, a)
gradient = cost_function.gradient(cost)
d_out = sigmoid_layer.backward(gradient)
d_in = linear_layer.backward(d_out)
linear_layer.update_parameters(learning_rate)

net = Network([linear_layer, sigmoid_layer])
out = net.forward(x=random_input)
cost = cost_function.estimate(random_label, out)
gradient = cost_function.gradient(cost)
d_in = net.backward(d=gradient)
net.update_parameters(learning_rate)

print("Task 4 output")
train(net, cost_function, number_epochs, batch_size, learning_rate)

print("Task 5 output")
batch_size = 600
learning_rate = 0.0001
number_epochs = 150
linear_layer1 = Linear(input_size, 300)
sigmoid_layer1 = Sigmoid()
linear_layer4 = Linear(300, label_size)
sigmoid_layer4 = Sigmoid()

cost_function = MeanSquareError()
net2 = Network([linear_layer1, sigmoid_layer1, linear_layer4, sigmoid_layer4])
train(net2, cost_function, number_epochs, batch_size, learning_rate)

/opt/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:373: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)] * n_features'.
  warnings.warn(msg, DeprecationWarning)
validation, e=0:  91%|█████████ | 31/34 [00:00<00:00, 144.37it/s]

cost=22556.975420, accuracy=0.455707



validation, e=1:  88%|████████▊ | 30/34 [00:00<00:00, 135.32it/s]

cost=17075.206707, accuracy=0.636566



validation, e=2:  94%|█████████▍| 32/34 [00:00<00:00, 151.05it/s]

...

cost=3097.883621, accuracy=0.928838



validation, e=148:  71%|███████   | 24/34 [00:00<00:00, 87.09it/s]

cost=3089.607925, accuracy=0.928889



validation, e=149:  71%|███████   | 24/34 [00:00<00:00, 89.77it/s]

cost=3081.392000, accuracy=0.928939
